<a href="https://colab.research.google.com/github/GeoKauko/TheNavySeals/blob/main/3_Postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Postprocessing

In [9]:
!pip install torch rasterio torchvision tifffile segmentation-models-pytorch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.0 MB/s eta 0:00:00


In [12]:
## Import packages
import os
import rasterio
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from PIL import Image

# we can maybe delete later:
import tifffile

In [17]:
## GOOGLE COLAB USERS ONLY
## Mount Google Drive for data retrieval

from google.colab import drive
drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/TheNavySeals/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
## LOCAL USERS ONLY
## Change the path to your project directory

os.chdir('D:\E_2024_P6\SEAL')

project_path = ''

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\E_2024_P6\\SEAL'

In [18]:
## Load functions from preprocessing
%run "1_Preprocessing.ipynb"

Exception: File `'1_Preprocessing.ipynb.py'` not found.

In [19]:
## Define paths
#source_image
reduced_image_path = 'data/3_postprocessing/3a_reduced_image'
os.makedirs(reduced_image_path, exist_ok=True)
reduced_image = os.path.join(reduced_image_path, '1a_reduced_image.tif')

tiled_images_path = 'data/3_postprocessing/3b_tiled_images'
os.makedirs(tiled_images_path, exist_ok=True)

predicted_masks_path = 'data/3_postprocessing/3c_predicted_masks'
os.makedirs(predicted_masks_path, exist_ok=True)

predicted_mask_final = 'data/3_postprocessing/3d_predicted_mask_final'
os.makedirs(predicted_mask_final, exist_ok=True)
predicted_mask = os.path.join(predicted_masks_path, '3d_predicted_mask.tif')

mask_heatmap_path = 'data/3_postprocessing/3e_mask_heatmap'
os.makedirs(predicted_mask_final, exist_ok=True)
mask_heatmap = os.path.join(predicted_masks_path, '3e_mask_heatmap.tif')

In [20]:
## Process the image
# Reduce image for increased processing speed
reduce_radiometric_resolution(source_image, reduced_image)
# Tile the image into 224x224 px tiles
split_and_save_raster(reduced_image, 224, 224, tiled_images_path)

NameError: name 'reduce_radiometric_resolution' is not defined

In [ ]:
def is_multiband_pil(image_path):
    """
    Check if an image is multiband or singleband using Pillow.

    Args:
        image_path (str): Path to the image file.

    Returns:
        bool: True if the image is multiband (color), False if it is singleband (grayscale).
    """
    try:
        with Image.open(image_path) as img:
            return img.mode in ("RGB", "RGBA", "CMYK", "YCbCr")
    except Exception as e:
        print(f"Error opening image: {e}")
        return False

## integrate model

In [25]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

def segmentation_dataset(data_path, transform=None):
   images_path = os.path.join(data_path, "images")  # We might be able to comment this out since the files are already loaded in this code
   image_files = os.listdir(images_path)

   dataset = []
   for img_name in image_files:
         image = tifffile.imread(os.path.join(images_path, img_name)) # We might be able to comment this out since the files are already loaded in this code

         if transform:
           image = transform(image)

         dataset.append((image)) #append the image-mask pair in the dataset

   return dataset



In [26]:
test_data_path = "/content/drive/MyDrive/SealNN/Pan_data/test/" # change this to other file

test_dataset = segmentation_dataset(test_data_path, transform=transform)

In [27]:
# Ceck the GPU colab assigns to you
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun 19 10:10:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              27W /  70W |    543MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
# Import model (the file path will be changed later)
model_name = "Model_TRIAL" # Make sure you change the name to the correct name
model_path = "/content/drive/MyDrive/SealNN/models/"+ model_name
model = torch.load(model_path)

In [29]:
# transfer model to cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [36]:
# Predict and save tiff file

pred_masks = []
for i in range(len(test_dataset)):
  image = test_dataset[i]
  image = image.to(device, dtype=torch.float).unsqueeze(0)  # Add batch dimension

  model.eval()
  with torch.no_grad():
    pred = model(image)

  pred = torch.sigmoid(pred).squeeze().cpu().numpy()  # Convert prediction to numpy array
  pred_print = (pred == 1).astype(float)

  pred_masks.append(pred_print)

  tifffile.imwrite(f'/content/drive/MyDrive/SealNN/pred/pred_mask_{i}.tiff', pred_print) # maybe change path



In [ ]:
## Process predicted masks into a heatmap
# Mosaic the masks back together
mosaic_rasters(predicted_masks_path, predicted_mask_final)

# Create a heatmap
def create_heatmap(input_path, output_path):
    """
    Save a heatmap from the given data to a file.

    Args:
        input_path (str): Path to the mask raster.
        output_path (str): Path to save the heatmap image.
    """
    with rasterio.open(input_path) as src:
        input_data = src.read(1)
    plt.figure(figsize=(10, 8))
    plt.imshow(input_data, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.title('Heatmap of mask.tif')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')

    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

create_heatmap(predicted_mask_final, mask_heatmap)